# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [21]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2460795486030,
 'week52high': 157.03,
 'week52low': 111.9,
 'week52highSplitAdjustOnly': 157.26,
 'week52lowSplitAdjustOnly': 112.59,
 'week52change': 0.2654255108230623,
 'sharesOutstanding': 16406397000,
 'float': 0,
 'avg10Volume': 56359970,
 'avg30Volume': 66965568,
 'day200MovingAvg': 141.04,
 'day50MovingAvg': 146.34,
 'employees': 147000,
 'ttmEPS': 11.23,
 'ttmDividendRate': 0.8630784880892401,
 'dividendYield': 0.005754240203275153,
 'nextDividendDate': '',
 'exDividendDate': '2021-11-05',
 'nextEarningsDate': '2022-01-26',
 'peRatio': 12.995328929182511,
 'beta': 1.3951008373548823,
 'maxChangePercent': 56.73731619062284,
 'year5ChangePercent': 5.02957906712173,
 'year2ChangePercent': 1.3182738527643396,
 'year1ChangePercent': 0.2654255108230623,
 'ytdChangePercent': 0.13734768003518427,
 'month6ChangePercent': 0.18033679614775355,
 'month3ChangePercent': 0.007437398703811882,
 'month1ChangePercent': 0.04485877792135873,
 'day30Chang

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [22]:
data['year1ChangePercent']

0.2654255108230623

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [3]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
# def chunks(lst, n):
#     """Yield successive n-sized chunks from lst."""
#     for i in range(0, len(lst), n):
#         yield lst[i:i + n]   
        
# symbol_groups = list(chunks(stocks['Ticker'], 3))
# symbol_strings = []
# for i in range(0, len(symbol_groups)):
#     symbol_strings.append(','.join(symbol_groups[i]))
    #print(symbol_strings[i])
for stock in stocks['Ticker']:
    print(stock)


my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

A
AAL
AAP
AAPL
ABBV
ABC
ABMD
ABT
ACN
ADBE


Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [25]:
final_dataframe = pd.DataFrame(columns = my_columns)
#final_dataframe
for symbol in stocks['Ticker']:
    api_call_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}'
    api_call_price =f'https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'

    data = requests.get(api_call_url).json()
    price= requests.get(api_call_price).json()
    #from printing we see a lot of nested info, we grab price and stats endpoint
#     print(data) #<-- looking in here, we see where to get the exact values
#     print(price)
    final_dataframe = final_dataframe.append(
    pd.Series( #this is to match up with the table columns
    [
        symbol,
        price['iexRealtimePrice'], 
        data['year1ChangePercent'],
        'N/A'
            
    ],
    index = my_columns),
    ignore_index = True
    )

final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,157.74,0.468662,N/A
1,AAL,20.31,0.657680,N/A
2,AAP,242.00,0.572326,N/A
3,AAPL,149.98,0.265426,N/A
4,ABBV,116.85,0.237687,N/A
5,ABC,124.38,0.176996,N/A
6,ABMD,348.36,0.326314,N/A
7,ABT,129.44,0.158397,N/A
8,ACN,368.36,0.554382,N/A
9,ADBE,659.73,0.401116,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [26]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True) #inplace replaces the table
final_dataframe = final_dataframe[:5] #i just want the top 50
final_dataframe.reset_index(inplace = True) #reset the index
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,1,AAL,20.31,0.657680,N/A
1,2,AAP,242.00,0.572326,N/A
2,8,ACN,368.36,0.554382,N/A
3,0,A,157.74,0.468662,N/A
4,9,ADBE,659.73,0.401116,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [9]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('enter portfolio size')
    try:
        float(portfolio_size)
    except ValueError:
        portfolio_size = input('enter a number')

portfolio_input()
print(portfolio_size)

enter portfolio size5000
5000


In [28]:
position_size = float(portfolio_size)/len(final_dataframe.index)
#print(position_size)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])

final_dataframe
#we've chosen the top 50 high movers and spent the money evenly between them
#no matter how much many stocks that means

/home/warrenball/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,1,AAL,20.31,0.657680,492
1,2,AAP,242.00,0.572326,41
2,8,ACN,368.36,0.554382,27
3,0,A,157.74,0.468662,63
4,9,ADBE,659.73,0.401116,15


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [4]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

#hqm_dataframe
#same logic as before, using batch calls for 
for symbol in stocks['Ticker']:
    api_call_change = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}'
    api_call_price =f'https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
    
    changes = requests.get(api_call_change).json()
    price = requests.get(api_call_price).json()
    
    hqm_dataframe = hqm_dataframe.append(
    pd.Series(
    [
        symbol, #'Ticker'
        price['iexRealtimePrice'], #'Price'
        'N/A',
        changes['year1ChangePercent'],
        'N/A',
        changes['month6ChangePercent'],
        'N/A',
        changes['month3ChangePercent'],
        'N/A',
        changes['month1ChangePercent'],
        'N/A',
        'N/A'
    ],
        index = hqm_columns),
        ignore_index = True
        )
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,157.74,N/A,0.468662,N/A,0.229829,N/A,0.011620,N/A,0.054329,N/A,N/A
1,AAL,20.31,N/A,0.657680,N/A,-0.094196,N/A,0.022681,N/A,0.033096,N/A,N/A
2,AAP,242.00,N/A,0.572326,N/A,0.190429,N/A,0.163614,N/A,0.112538,N/A,N/A
3,AAPL,149.98,N/A,0.265426,N/A,0.180337,N/A,0.007437,N/A,0.044859,N/A,N/A
4,ABBV,116.85,N/A,0.237687,N/A,0.028153,N/A,0.016381,N/A,0.074006,N/A,N/A
5,ABC,124.38,N/A,0.176996,N/A,0.083619,N/A,0.052107,N/A,0.047377,N/A,N/A
6,ABMD,348.36,N/A,0.326314,N/A,0.343797,N/A,0.121996,N/A,0.045846,N/A,N/A
7,ABT,129.44,N/A,0.158397,N/A,0.094048,N/A,0.047830,N/A,0.092541,N/A,N/A
8,ACN,368.36,N/A,0.554382,N/A,0.295569,N/A,0.151127,N/A,0.094945,N/A,N/A
9,ADBE,659.73,N/A,0.401116,N/A,0.351529,N/A,0.031837,N/A,0.082451,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [5]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'    
]
#we're saving these values from the main list to help

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.fillna(0,inplace=True)# need this if batch call gives any n/a
        
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        #score = percentileofscore, put in column and value
        hqm_dataframe.loc[row, percentile_col] = score( hqm_dataframe[change_col] , hqm_dataframe.loc[row, change_col] )/100
        #divide by 100 to make it a percentage
        #print(type(hqm_dataframe[f'{time_period} Price Return']))
        #print((hqm_dataframe.loc[row, f'{time_period} Price Return']))
        #BE Careful!! in four of these calls, there are nontypes rather than floats, the print statements above reveal it
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,157.74,N/A,0.468662,0.7,0.229829,0.7,0.011620,0.2,0.054329,0.5,N/A
1,AAL,20.31,N/A,0.657680,1.0,-0.094196,0.1,0.022681,0.4,0.033096,0.1,N/A
2,AAP,242.00,N/A,0.572326,0.9,0.190429,0.6,0.163614,1.0,0.112538,1.0,N/A
3,AAPL,149.98,N/A,0.265426,0.4,0.180337,0.5,0.007437,0.1,0.044859,0.2,N/A
4,ABBV,116.85,N/A,0.237687,0.3,0.028153,0.2,0.016381,0.3,0.074006,0.6,N/A
5,ABC,124.38,N/A,0.176996,0.2,0.083619,0.3,0.052107,0.7,0.047377,0.4,N/A
6,ABMD,348.36,N/A,0.326314,0.5,0.343797,0.9,0.121996,0.8,0.045846,0.3,N/A
7,ABT,129.44,N/A,0.158397,0.1,0.094048,0.4,0.047830,0.6,0.092541,0.8,N/A
8,ACN,368.36,N/A,0.554382,0.8,0.295569,0.8,0.151127,0.9,0.094945,0.9,N/A
9,ADBE,659.73,N/A,0.401116,0.6,0.351529,1.0,0.031837,0.5,0.082451,0.7,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [6]:
from statistics import mean

#now we grab the means of all the return percentiles

for row in hqm_dataframe.index:
    momentum_percentiles = [] #save all percentile scores in this list and grab the mean of all of it
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe    

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,157.74,N/A,0.468662,0.7,0.229829,0.7,0.011620,0.2,0.054329,0.5,0.525
1,AAL,20.31,N/A,0.657680,1.0,-0.094196,0.1,0.022681,0.4,0.033096,0.1,0.4
2,AAP,242.00,N/A,0.572326,0.9,0.190429,0.6,0.163614,1.0,0.112538,1.0,0.875
3,AAPL,149.98,N/A,0.265426,0.4,0.180337,0.5,0.007437,0.1,0.044859,0.2,0.3
4,ABBV,116.85,N/A,0.237687,0.3,0.028153,0.2,0.016381,0.3,0.074006,0.6,0.35
5,ABC,124.38,N/A,0.176996,0.2,0.083619,0.3,0.052107,0.7,0.047377,0.4,0.4
6,ABMD,348.36,N/A,0.326314,0.5,0.343797,0.9,0.121996,0.8,0.045846,0.3,0.625
7,ABT,129.44,N/A,0.158397,0.1,0.094048,0.4,0.047830,0.6,0.092541,0.8,0.475
8,ACN,368.36,N/A,0.554382,0.8,0.295569,0.8,0.151127,0.9,0.094945,0.9,0.85
9,ADBE,659.73,N/A,0.401116,0.6,0.351529,1.0,0.031837,0.5,0.082451,0.7,0.7


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [7]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:5]# We now sort based on how high the HQM score is
hqm_dataframe.reset_index(drop = True, inplace = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,AAP,242.00,N/A,0.572326,0.9,0.190429,0.6,0.163614,1.0,0.112538,1.0,0.875
1,ACN,368.36,N/A,0.554382,0.8,0.295569,0.8,0.151127,0.9,0.094945,0.9,0.85
2,ADBE,659.73,N/A,0.401116,0.6,0.351529,1.0,0.031837,0.5,0.082451,0.7,0.7
3,ABMD,348.36,N/A,0.326314,0.5,0.343797,0.9,0.121996,0.8,0.045846,0.3,0.625
4,A,157.74,N/A,0.468662,0.7,0.229829,0.7,0.011620,0.2,0.054329,0.5,0.525


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [10]:
portfolio_input()

enter portfolio size5000


In [11]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
#position_size

for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price']) 

hqm_dataframe

/home/warrenball/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,AAP,242.00,4,0.572326,0.9,0.190429,0.6,0.163614,1.0,0.112538,1.0,0.875
1,ACN,368.36,2,0.554382,0.8,0.295569,0.8,0.151127,0.9,0.094945,0.9,0.85
2,ADBE,659.73,1,0.401116,0.6,0.351529,1.0,0.031837,0.5,0.082451,0.7,0.7
3,ABMD,348.36,2,0.326314,0.5,0.343797,0.9,0.121996,0.8,0.045846,0.3,0.625
4,A,157.74,6,0.468662,0.7,0.229829,0.7,0.011620,0.2,0.054329,0.5,0.525


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [16]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = "Momentum Strategy", index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [17]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [18]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column( f'{column}:{column}', 22, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0] , column_formats[column][1])


## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [19]:
writer.save() #oddly enough, it won't update in your system, you must download the excel file